In [1]:
import keras
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
import os
import pickle
import numpy as np
import nibabel as nib

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
img = nib.load('brainmask_fs.nii')

In [4]:
img.shape

(260, 311, 260)

In [5]:
data = img.get_data()
data.shape

(260, 311, 260)

In [8]:
print data.sum()

4515185.0


In [41]:
x_train = data.reshape(1,260,260,311)

In [42]:
# normalize data
x_train = x_train.astype('float32')
x_train /= 255.

In [47]:
input_img = Input(shape=(260, 260, 311))
x = Conv2D(64, (3, 3), padding='same')(input_img)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(32, (3, 3), padding='same')(encoded)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(311, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
decoded = Activation('sigmoid')(x)

In [48]:
model = Model(input_img, decoded)
model.compile(optimizer='adam', loss='binary_crossentropy')

In [49]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 260, 260, 311)     0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 260, 260, 64)      179200    
_________________________________________________________________
batch_normalization_57 (Batc (None, 260, 260, 64)      256       
_________________________________________________________________
activation_57 (Activation)   (None, 260, 260, 64)      0         
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 130, 130, 64)      0         
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 130, 130, 32)      18464     
_________________________________________________________________
batch_normalization_58 (Batc (None, 130, 130, 32)      128       
__________

In [50]:
history = model.fit(x_train, x_train,
                    batch_size=1,
                    epochs=100,
                    verbose=1,
                    shuffle=False)

Epoch 1/100
1/1 [==============================] - 6s 6s/step - loss: 0.8001
Epoch 2/100
1/1 [==============================] - 4s 4s/step - loss: 0.7963
Epoch 3/100
1/1 [==============================] - 4s 4s/step - loss: 0.7918
Epoch 4/100
1/1 [==============================] - 5s 5s/step - loss: 0.7852
Epoch 5/100
1/1 [==============================] - 5s 5s/step - loss: 0.7782
Epoch 6/100
1/1 [==============================] - 4s 4s/step - loss: 0.7702
Epoch 7/100
1/1 [==============================] - 4s 4s/step - loss: 0.7639
Epoch 8/100
1/1 [==============================] - 4s 4s/step - loss: 0.7588
Epoch 9/100
1/1 [==============================] - 4s 4s/step - loss: 0.7536
Epoch 10/100
1/1 [==============================] - 5s 5s/step - loss: 0.7494
Epoch 11/100
1/1 [==============================] - 4s 4s/step - loss: 0.7460
Epoch 12/100
1/1 [==============================] - 5s 5s/step - loss: 0.7432
Epoch 13/100


KeyboardInterrupt: 